# K8S Section
### config

In [58]:
import base64
import yaml
import re

INPUT_FILE = 'io/props/edit.properties'
OUTPUT_PATH = 'io/k8s/'
TEMPLATE_PATH = 'io/templates/'

CONFIGMAP = 'configmap.yaml'
DEPLOYMENT = 'deployment.yaml'
SECRET = 'secret.yaml'
SERVICE = 'service.yaml'

### templates

In [59]:
def find_placeholders(template:str) -> set :
    pattern = re.compile(r'\{(.*?)\}')
    placeholders = pattern.findall(template)
    return set(placeholders)

def replace_placeholders(template:str, values:dict) -> str:
    pattern = re.compile(r'\{(.*?)\}')
    result = template
    for match in pattern.findall(template):
        key = match.strip()
        if key in values:
            result = result.replace(f'{{{key}}}', values[key])
    return result

In [60]:
#load templates and find variables
with open(TEMPLATE_PATH+CONFIGMAP, 'r') as file:
    configmap_template = file.read()
with open(TEMPLATE_PATH+DEPLOYMENT, 'r') as file:
    deployment_template = file.read()
with open(TEMPLATE_PATH+SECRET, 'r') as file:
    secret_template = file.read()
with open(TEMPLATE_PATH+SERVICE, 'r') as file:
    service_template = file.read()

placeholders = find_placeholders(deployment_template + configmap_template + secret_template + service_template)
placeholders_values = dict.fromkeys(placeholders, '')
print('Fill the following dict in the next cell')
placeholders_values

Fill the following dict in the next cell


{'SERVICE_NAME': '',
 'NAMESPACE': '',
 'IMAGE': '',
 'DEPLOYMENT_NAME': '',
 'SECRET_NAME': '',
 'TARGET_PORT': '',
 'CONFIGMAP_NAME': '',
 'PORT': ''}

In [61]:
# copy the last output here and give them a value
placeholders_values = {
    'DEPLOYMENT_NAME' : 'example_app',
    'SERVICE_NAME' : 'example_app_service',
    'NAMESPACE' : 'test',
    'IMAGE' : 'example:lastest',
    'SECRET_NAME' : 'example_app_secret',
    'TARGET_PORT' : '8088',
    'CONFIGMAP_NAME' : 'example_app_env',
    'PORT' : '8088'
}

configmap_inner = replace_placeholders(configmap_template, placeholders_values)
secret_inner = replace_placeholders(secret_template, placeholders_values)
service_inner = replace_placeholders(service_template, placeholders_values)
deployment_inner = replace_placeholders(deployment_template, placeholders_values)

### write

In [62]:
configmap_out = yaml.safe_load(configmap_inner)
secrets_out = yaml.safe_load(secret_inner)
service_out = yaml.safe_load(service_inner)
deployment_out = yaml.safe_load(deployment_inner)

with open(INPUT_FILE, 'r') as f:
    lines = f.readlines()
    secrets = dict()
    configs = dict()
    for line in lines:
        if line.startswith('#') or line=='':
            continue

        parts = line.strip().split('=')
        if len(parts) == 4 and parts[0] == 'S':
            _, config, _, value = parts
            secrets[config] = base64.b64encode(value.encode('ascii') ).decode('ascii')

        elif len(parts) == 3:
            config, _, value = parts
            configs[config] = value
                
    secrets_out['data'] = secrets
    configmap_out['data'] = configs

f.close()

with open(OUTPUT_PATH+CONFIGMAP, 'w+') as outfile:
    yaml.dump(configmap_out, outfile, default_flow_style=False, sort_keys=False)
with open(OUTPUT_PATH+SECRET, 'w+') as outfile:
    yaml.dump(secrets_out, outfile, default_flow_style=False, sort_keys=False)
with open(OUTPUT_PATH+DEPLOYMENT, 'w+') as outfile:
    yaml.dump(deployment_out, outfile, default_flow_style=False, sort_keys=False)
with open(OUTPUT_PATH+SERVICE, 'w+') as outfile:
    yaml.dump(service_out, outfile, default_flow_style=False, sort_keys=False)